In [14]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [27]:
df = pd.read_csv('Beach_Water_Samples_20240210.csv')

In [28]:
df

,Sample ID,Sample Date,Beach Name,Sample Location,Enterococci Results,Units or Notes
0,050514CP13,05/05/2014,MIDLAND BEACH,Center,20.0,MPN/100 ml
1,062011GR04,06/20/2011,MANHATTAN BEACH,Left,NaN,Result below detection limit
2,072808BH09,07/28/2008,MIDLAND BEACH,Right,28.0,MPN/100 ml
3,051214CP36,05/12/2014,SOUTH BEACH,Right,4.0,MPN/100 ml
4,081511KB07,08/15/2011,CEDAR GROVE,Left,360.0,MPN/100 ml
...,...,...,...,...,...,...
26994,MB2309050945-1.1,09/05/2023,MANHATTAN BEACH,Left,9.9,MPN/100 ml
26995,JB2309070930-3.1,09/07/2023,ROCKAWAY BEACH 23RD - 59TH,Center,30.0,MPN/100 ml
26996,MB2309061010-1.3,09/06/2023,SCHUYLER HILL CIVIC ASSOCIATION,Right,10.0,MPN/100 ml
26997,AC2309060850-1.1,09/06/2023,ORCHARD BEACH,Left,9.9,MPN/100 ml


In [30]:
df['Sample Date'] = pd.to_datetime(df['Sample Date'], format='%m/%d/%Y')

In [32]:
df.head()

,Sample ID,Sample Date,Beach Name,Sample Location,Enterococci Results,Units or Notes
0,050514CP13,2014-05-05,MIDLAND BEACH,Center,20.0,MPN/100 ml
1,062011GR04,2011-06-20,MANHATTAN BEACH,Left,NaN,Result below detection limit
2,072808BH09,2008-07-28,MIDLAND BEACH,Right,28.0,MPN/100 ml
3,051214CP36,2014-05-12,SOUTH BEACH,Right,4.0,MPN/100 ml
4,081511KB07,2011-08-15,CEDAR GROVE,Left,360.0,MPN/100 ml
